# 🧪 WFGY Diagnostic Notebook (v0.2)
**Minimal sandbox to measure ΔS and spot common reasoning failures (Problem Map v1.0).**  
Runs entirely in Colab — no API key, no local install.

## 📐 Core formula
$$\Delta S \;=\; 1 - \cos\theta\,\bigl(I,\,G\bigr) \qquad \bigl(I,\,G \in \mathbb R^{1536}\bigr)$$
*$I$ = current-step embedding,  $G$ = ground-truth / prompt embedding.*  
Higher ΔS ⇒ larger semantic drift ⇒ higher chance of collapse.

## ▶️ Quick Start
1. **Run all** cells (≃ 40 s).  
2. Click **Run demo** or paste your own prompt / answer.  
3. Get ΔS, failure type, and suggested WFGY fix.

In [ ]:
!pip -q install sentence-transformers ipywidgets --upgrade

In [ ]:
from sentence_transformers import SentenceTransformer, util
from ipywidgets import Textarea, Button, VBox, HBox, Layout, Output
import math, textwrap, IPython.display as disp

In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')  # small & fast

In [ ]:
def delta_s(a_vec, b_vec):
    cos_sim = util.cos_sim(a_vec, b_vec).item()
    return 1 - cos_sim

def classify_failure(dS):
    if dS > 0.80:
        return ('No.1 Hallucination / Chunk Drift', 'Apply BBMC + boundary check')
    if dS > 0.60:
        return ('No.2 Interpretation Collapse', 'Insert λ_observe checkpoint')
    if dS > 0.40:
        return ('No.6 Logic Collapse', 'Trigger BBCR fallback')
    if dS > 0.25:
        return ('No.9 Entropy Collapse', 'Use BBAM attention modulation')
    return ('Stable (ΔS ≤ 0.25)', 'No action — pass')

In [ ]:
demo_prompt = "### Product Review\nThe new phone has a crystal-clear display and fast charging…"
demo_answer = "The capital of France is Paris. Also, your phone seems great!"

p_box = Textarea(value=demo_prompt, placeholder='Paste prompt / context', description='Prompt', layout=Layout(width='100%', height='100px'))
a_box = Textarea(value=demo_answer, placeholder='Paste LLM answer', description='Answer', layout=Layout(width='100%', height='100px'))
out = Output()

btn = Button(description='Run diagnostic', button_style='primary', tooltip='Compute ΔS & classify failure')

def on_click(b):
    out.clear_output()
    with out:
        p_vec = model.encode(p_box.value, convert_to_tensor=True)
        a_vec = model.encode(a_box.value, convert_to_tensor=True)
        dS = delta_s(p_vec, a_vec)
        failure, fix = classify_failure(dS)
        print('ΔS :', f'{dS:.3f}')
        print('Failure detected :', failure)
        print('Suggested fix   :', fix)

btn.on_click(on_click)
disp.display(VBox([p_box, a_box, btn, out]))

---
### Next Steps
* Explore the full 16-type **Problem Map 1.0** and the RAG-focused **Problem Map 2.0**.  
* Read about λ_observe, E_resonance, BBCR in the **WFGY 1.0 paper**.  
* Fork this notebook, plug in your embeddings or extend the rules — it’s MIT-licensed.

> Repo → https://github.com/onestardao/WFGY